In [1]:
%reset -sf
import numpy as np
import os
from tqdm import tqdm
import multiprocessing as mp
from multiprocessing import Pool

In [2]:
os.chdir("/home/pj/projects/clifford_alg/data")
cliffords2 = np.load('cliffords_2Qnp.npy')
rtol=1e-03
atol=1e-05

In [3]:
def flatten_and_uniquenize(clifford):
    flattened = clifford.ravel('C')
    for j in range(len(flattened)):
        if not np.isclose(0, flattened[j]):
            normalizer = np.conj(flattened[j]) / np.abs(flattened[j])
            break
        else:
            continue
    flattened = flattened * normalizer
    return flattened

In [4]:
flatten_and_uniquenize(cliffords2[5])

array([ 7.07106781e-01+0.00000000e+00j, -1.66533454e-16+7.07106781e-01j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        7.07106781e-01+1.66533454e-16j,  3.33066907e-16-7.07106781e-01j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        7.07106781e-01+0.00000000e+00j, -1.66533454e-16+7.07106781e-01j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        7.07106781e-01+1.66533454e-16j,  3.33066907e-16-7.07106781e-01j])

In [5]:
uniqueCliffords = []
for i in tqdm(range(len(cliffords2))):
    uniqueCliffords.append(flatten_and_uniquenize(cliffords2[i]))
uniqueCliffords = np.asarray(uniqueCliffords).ravel('C')

100%|██████████| 11520/11520 [00:00<00:00, 16244.75it/s]


In [6]:
def uniquenize_elements(flatarray):
    for i in tqdm(range(len(flatarray))):
        for j in range(i, len(flatarray)):
            if np.isclose(flatarray[i], flatarray[j], rtol, atol):
                flatarray[j] = flatarray[i]
    return np.unique(flatarray)

In [7]:
uniqueElements = np.unique(uniqueCliffords)
uniqueElements = uniquenize_elements(uniqueElements)

print("elements of 2d-clifford_group:{}\nunique elements:{}".format(len(cliffords2), len(uniqueElements)))

100%|██████████| 1369/1369 [00:23<00:00, 58.54it/s]

elements of 2d-clifford_group:11520
unique elements:13


In [8]:
cliffordsHash = []
for clifford in tqdm(cliffords2):
    flattened = flatten_and_uniquenize(clifford)
    cliffordHash = np.zeros(len(flattened), dtype = 'uint8')
    for i in range(len(flattened)):
        for j in range(len(uniqueElements)):
            if np.isclose(uniqueElements[j], flattened[i], rtol, atol):
                cliffordHash[i] = j + 1
                break;
    cliffordsHash.append(cliffordHash)

100%|██████████| 11520/11520 [00:36<00:00, 314.19it/s]


In [9]:
def hashnize(flattened, elementsHash):
    ret = np.zeros(len(flattened), dtype='int8')
    for i in range(len(flattened)):
        for j in range(len(elementsHash)):
            if np.isclose(elementsHash[j], flattened[i], rtol, atol):
                ret[i] = j + 1
                break
    return ret

In [10]:
def clifford_to_hash(clifford, elementHash):
    return hashnize(flatten_and_uniquenize(clifford), uniqueElements)

In [11]:
print("C4:\n {}\nhash: \n{}".format(cliffords2[4], clifford_to_hash(cliffords2[4], uniqueElements)))

C4:
 [[ 0.5-0.5j  0.5+0.5j  0. +0.j   0. +0.j ]
 [-0.5+0.5j  0.5+0.5j  0. +0.j   0. +0.j ]
 [ 0. +0.j   0. +0.j   0.5-0.5j  0.5+0.5j]
 [ 0. +0.j   0. +0.j  -0.5+0.5j  0.5+0.5j]]
hash: 
[12  6 10 10  2  6 10 10 10 10 12  6 10 10  2  6]


In [12]:
def prod(index):
    # index = [i, j]
    producted = np.matmul(cliffords2[index[0]], cliffords2[index[1]])
    productedHash = hashnize(flatten_and_uniquenize(producted), uniqueElements)
    for k in range(len(cliffordsHash)):
        if np.array_equal(cliffordsHash[k], productedHash):
            map[index[0]][index[1]] = k + 1
            return
        else:
            continue

In [16]:
L = 10
map = np.zeros((L, L), dtype='int8')
def calc():
    p = Pool(mp.cpu_count())
    arg = [[i, j] for i in range(L) for j in range(L)]
    p.map(prod, arg)
    p.close()
    return

In [17]:
calc()
map

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int8)

In [25]:
np.savetxt("cliffords.csv", map, delimiter=',' ,fmt = '%i')

In [18]:
L = 10
map = np.zeros((L, L), dtype='int8')
for i in tqdm(range(L)):
    for j in range(L):
        producted = np.matmul(cliffords2[i], cliffords2[j])
        productedHash = hashnize(flatten_and_uniquenize(producted), uniqueElements)
        for k in range(len(cliffordsHash)):
            if np.array_equal(cliffordsHash[k], productedHash):
                map[i][j] = k + 1
                break
            else:
                continue

100%|██████████| 10/10 [00:00<00:00, 25.22it/s]


In [19]:
map

array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       [ 2,  1,  4,  3,  7,  8,  5,  6, 12, 11],
       [ 3,  4,  1,  2,  8,  7,  6,  5, 11, 12],
       [ 4,  3,  2,  1,  6,  5,  8,  7, 10,  9],
       [ 5,  8,  6,  7, 12,  9, 10, 11,  3,  4],
       [ 6,  7,  5,  8, 11, 10,  9, 12,  2,  1],
       [ 7,  6,  8,  5,  9, 12, 11, 10,  4,  3],
       [ 8,  5,  7,  6, 10, 11, 12,  9,  1,  2],
       [ 9, 10, 12, 11,  2,  4,  3,  1,  8,  5],
       [10,  9, 11, 12,  3,  1,  2,  4,  7,  6]], dtype=int8)